In [ ]:
!pip install pyresparser
!pip install nltk
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 4.8 MB/s eta 0:00:00
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3959 sha256=1fc5e03d4fb320208e0c9e4ef0addac49bc8310d045eed4d4ce1fe45235b4020
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 70.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 369.2/369.2 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.3 MB/s eta 0:00:00
  Cre

In [ ]:
import nltk
nltk.download('stopwords')

from pyresparser import ResumeParser
import warnings

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyresparser import ResumeParser
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

data = ResumeParser("/content/drive/MyDrive/MyResume.pdf").get_extracted_data()

print("Name:", data["name"])
print("Email:", data["email"])
print("Mobile Number:", data["mobile_number"])
print("Skills:", data["skills"])
print("Degree:", data["degree"])
print("No Of Pages:", data["no_of_pages"])
print("Experience:", data["experience"])
print(data)

{'name': 'Shiv Rakh', 'email': 'shiv.rakh@vit.edu.in', 'mobile_number': '9321611060', 'skills': ['Javascript', 'Database', 'Engineering', 'Communication', 'C', 'Technical skills', 'Github', 'Postgresql', 'Api', 'Java', 'Cloud', 'Aws', 'Certification', 'Apis', 'Css', 'C++', 'Python', 'Technical', 'Opencv', 'Html', 'System'], 'college_name': None, 'degree': ['Bachelor of Engineering - Computer Engineering'], 'designation': ['Computer Engineer'], 'experience': ['personalized note-taking.', 'FetchJoke Web App', 'Description: A web application using public APIs to fetch jokes', 'Technologies: HTML, CSS, JavaScript, Node.js', 'Responsibilities:', '• Integrated public APIs.', '• User-friendly interfaces.', '• Handled API key management and data fetching.', 'Key Achievements: Created an engaging app enhancing user interaction and entertainment.'], 'company_names': ['TCS iON Career'], 'no_of_pages': 1, 'total_experience': 0.0}


In [ ]:
import pandas as pd


In [ ]:
csv_data = {
    'Name': [data['name']],
    'Skills': [', '.join(data['skills'])]
}


df = pd.DataFrame(csv_data)


df.to_csv('resume_data.csv', index=False)

print("CSV file created successfully!")

CSV file created successfully!


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the CSV files
resume_data = pd.read_csv('/content/random_applicants_with_skills.csv')  # Assume it has a 'Skills' column
company_data = pd.read_csv('/content/company_skills.csv')  # Assume it has 'Company Name' and 'Skills Required'




In [ ]:
# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Prepare a DataFrame to hold the best fits
best_fit_results = []

# Iterate over each applicant in the resume data
for index, applicant in resume_data.iterrows():
    applicant_skills = applicant['Skills']

    # Create a list of all skills for the TF-IDF vectorizer
    documents = [applicant_skills] + company_data['Skills Required'].tolist()

    # Fit the TF-IDF vectorizer
    tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

    # Calculate cosine similarity
    cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])

    # Create a DataFrame for results based on cosine similarity
    results_cosine = pd.DataFrame({
        'Company Name': company_data['Company Name'],
        'Cosine Similarity Score': cosine_sim.flatten()
    })

    # Sort results by cosine similarity score in descending order
    best_fit_company_cosine = results_cosine.sort_values(by='Cosine Similarity Score', ascending=False).iloc[0]

    # Extract TF-IDF scores for each company (excluding the applicant)
    tfidf_scores = pd.DataFrame(tfidf_matrix[1:].toarray(), columns=tfidf_vectorizer.get_feature_names_out(), index=company_data['Company Name'])

    # Sum the TF-IDF scores for each company to get a single score
    tfidf_scores['TF-IDF Score'] = tfidf_scores.sum(axis=1)

    # Get the best fit company based on the highest TF-IDF score
    best_fit_company_tfidf = tfidf_scores['TF-IDF Score'].idxmax()

    # Append the result to the list
    best_fit_results.append({
        'Applicant': applicant['Name'],  # Assuming there's a 'Name' column
        'Best Fit Company (Cosine)': best_fit_company_cosine['Company Name'],
        'Best Fit Company (TF-IDF)': best_fit_company_tfidf,
        'Cosine Similarity Score': best_fit_company_cosine['Cosine Similarity Score'],
        'TF-IDF Score': tfidf_scores.loc[best_fit_company_tfidf, 'TF-IDF Score']
    })

# Convert results to a DataFrame
best_fit_df = pd.DataFrame(best_fit_results)

# Display the results
print(best_fit_df[['Applicant', 'Best Fit Company (Cosine)', 'Best Fit Company (TF-IDF)', 'Cosine Similarity Score', 'TF-IDF Score']])

# Save the results to a CSV file
best_fit_df.to_csv('best_fit_companies_separate.csv', index=False)

         Applicant Best Fit Company (Cosine) Best Fit Company (TF-IDF)  \
0      Penny Davis                 Company A                 Company B   
1        Chris Lee                 Company F                 Company B   
2        Eva Green                 Company A                 Company B   
3    Rita Anderson                 Company C                 Company B   
4     Rachel Scott                 Company H                 Company B   
5     Quincy White                 Company H                 Company B   
6      Frank White                 Company F                 Company B   
7        Paul King                 Company F                 Company H   
8       Leo Harris                 Company B                 Company H   
9    Gabe Williams                 Company H                 Company B   
10  Henry Martinez                 Company J                 Company B   
11    Olivia Young                 Company B                 Company H   
12       Chris Lee                 Com